<a href="https://colab.research.google.com/github/QwertyApoorv987/Object_Detection/blob/main/Collecting_Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
DRIVER_PATH =webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (122 kB/s

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
import requests
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO
import cv2
import urllib.request
import numpy as np
search_term = ["twoanimals"] ###### change
number_images = 15
target_path = './images'


# !zip -r mouseandcats.zip mouseandcats/ 
# files.download('mouseandcats.zip') 

# !zip -r monkeys.zip monkeys/ 
# files.download('monkeys.zip') 

# !zip -r dogsandmouse.zip dogsandmouse/ 
# files.download('dogsandmouse.zip')

# !zip -r monkeysandcats.zip mouseandcats/ 
# files.download('monkeysandcats.zip')

# !zip -r monkeysanddogs.zip mouseanddogs/ 
# files.download('monkeysanddogs.zip')

# !zip -r monkeysandmouse.zip mouseandmouse/ 
# files.download('monkeysandmouse.zip')

In [ ]:
def url_to_image(url):
	# download the image, convert it to a NumPy array, and then read
	# it into OpenCV format
	resp = urllib.request.urlopen(url)
	image = np.asarray(bytearray(resp.read()), dtype="uint8")
	image = cv2.imdecode(image, cv2.IMREAD_COLOR)
	# return the image
	return image

In [ ]:
def get_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [ ]:
# def persist_image(folder_path:str,url:str,fname:int):
#   print(fname)
#   #img = url_to_image(url)
#   image = requests.get(url)
#   image.raise_for_status()  #### 2 steps for compplete returning of object from the url
#   img = Image.open(BytesIO(image.content)).convert('RGBA')
#   #all_images.append(img)
#   if len(str(fname))==3:
#     with open(str(fname) + '.jpg', 'wb') as f:
#       xyz = bytearray(image.content)
#       f.write(xyz)
#       fname = fname + 1
#   if len(str(fname))==2:
#     with open('0' + str(fname) + '.jpg', 'wb') as f:
#       xyz = bytearray(image.content)
#       f.write(xyz)
#       fname = fname + 1
#   if len(str(fname))==1:
#     with open('00' + str(fname) + '.jpg', 'wb') as f:
#       xyz = bytearray(image.content)
#       f.write(xyz)
#       fname = fname + 1

In [ ]:
def search_and_download(search_term:str, wd, target_path,number_images):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    #if not os.path.exists(target_folder):
    #    os.makedirs(target_folder)

    with wd as wd:
        res = get_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        #print(res)

    fname = 0   
    for elem in res:
      try:
        image = requests.get(elem)
        image.raise_for_status()  
        img = Image.open(BytesIO(image.content)).convert('RGBA')
        if len(str(fname))==3:
          with open(str(fname) + '.jpg', 'wb') as f:
            xyz = bytearray(image.content)
            f.write(xyz)
            fname = fname + 1
        if len(str(fname))==2:
          with open('0' + str(fname) + '.jpg', 'wb') as f:
            xyz = bytearray(image.content)
            f.write(xyz)
            fname = fname + 1
        if len(str(fname))==1:
          with open('00' + str(fname) + '.jpg', 'wb') as f:
            xyz = bytearray(image.content)
            f.write(xyz)
            fname = fname + 1
      except Exception as e:
        print("ERROR")
      #print(0)
        #persist_image(target_folder,elem,i)

In [ ]:
import os 
import time
import requests
import io
from PIL import Image
import hashlib
i= 0
!mkdir animals
%cd animals
for ele in search_term:
    search_and_download(ele, wd, target_path, number_images)
%cd ..

mkdir: cannot create directory ‘animals’: File exists
/content/animals
Found: 100 search results. Extracting links from 0:100
Found: 15 image links, done!
/content


In [ ]:
from google.colab import files                     ############################## change ###################
# #!zip -r mouseandcats.zip mouseandcats/ 
# #files.download('mouseandcats.zip') #### cat 

!zip -r animals.zip animals/ 
files.download('animals.zip') #### cat 

updating: animals/ (stored 0%)
updating: animals/013.jpg (deflated 0%)
updating: animals/006.jpg (deflated 0%)
updating: animals/001.jpg (deflated 1%)
updating: animals/008.jpg (deflated 0%)
updating: animals/007.jpg (deflated 3%)
updating: animals/010.jpg (deflated 0%)
updating: animals/011.jpg (deflated 0%)
updating: animals/003.jpg (deflated 2%)
updating: animals/014.jpg (deflated 1%)
updating: animals/005.jpg (deflated 1%)
updating: animals/004.jpg (deflated 1%)
updating: animals/000.jpg (deflated 2%)
updating: animals/012.jpg (deflated 3%)
updating: animals/002.jpg (deflated 3%)
updating: animals/009.jpg (deflated 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>